In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
from copy import copy
%matplotlib inline

class Lane():
    def __init__(self):
        # was the line detected in the last frame or not
        self.detected = False
        #x values for detected line pixels
        self.cur_fitx = None
        #y values for detected line pixels
        self.cur_fity = None
        # x values of the last N fits of the line
        self.prev_fitx = []
        #polynomial coefficients for the most recent fit
        self.current_poly = [np.array([False])]
        #best polynomial coefficients for the last iteration
        self.prev_poly = [np.array([False])]

    def average_pre_lanes(self):
        tmp = copy(self.prev_fitx)
     
        tmp.append(self.cur_fitx)
        
        self.mean_fitx = np.mean(tmp,axis=0)

    def append_fitx(self):
        if len(self.prev_fitx) == 4:# buffer previous N lines
            self.prev_fitx.pop(0)
        self.prev_fitx.append(self.mean_fitx)

    def process(self, ploty):
        self.cur_fity = ploty
        self.average_pre_lanes()
        self.append_fitx()
        self.prev_poly = self.current_poly
        
left_lane = Lane()
right_lane = Lane()

def calibration(images):
    objpoints = [] 
    imgpoints = []

    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    
    

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)


    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return mtx, dist

def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return binary_output

def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

def warp(img):
    img_size = (img.shape[1], img.shape[0])

    src = np.float32(
        [[194, 719],
         [1117, 719],
         [705, 461],
         [575, 461]])
        
    dst = np.float32(
        [[290, 719],
         [990, 719],
         [990, 0],
         [290, 0]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, Minv


def draw_lanes(image, bird_view, left_fitx, right_fitx, ploty, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(bird_view).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
        
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 1, 0)
    
    return result

def combined(image):
    grad_x = abs_sobel_thresh(image, orient='x', thresh_min=20, thresh_max=100)
           
    mag_binary = mag_thresh(image, sobel_kernel=5, mag_thresh=(35, 100))
           
    thresh=(90, 255)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
        
    combined = np.zeros_like(s_channel)
    
    combined[(grad_x == 1) & (mag_binary == 1)] = 1
        
    gradient_color_combined = np.zeros_like(s_channel)
    
    gradient_color_combined[(combined == 1) | (binary_output == 1)] = 1
    
    c_bi = np.zeros_like(s_channel)
    
    c_bi[(grad_x == 1) & (binary_output == 1) ] = 2
    
    ave_binary = (gradient_color_combined + c_bi)
    return gradient_color_combined

def cal_lane(histogram,bird_view): 
        
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    nwindows = 9
    window_height = np.int(bird_view.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = bird_view.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = bird_view.shape[0] - (window+1)*window_height
        win_y_high = bird_view.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, bird_view.shape[0]-1, bird_view.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
    return left_fitx, right_fitx, ploty, left_fit, right_fit, leftx, rightx
        
          
def cal_curvature(image, ploty, leftx, rightx):
   
    leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  # Reverse to match top-to-bottom in y
     # choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)    
    # Define conversions in x and y from pixels space to meters
    
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
   
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    middle_of_the_lanes =  (leftx[-1] + rightx[-1])/2.0
    vehicle_position = image.shape[1]/2.0
    
    distance = np.absolute((vehicle_position - middle_of_the_lanes)*xm_per_pix)
    
    if (vehicle_position - middle_of_the_lanes) < 0:
        cv2.putText(image,'Left radius of curvature  = %.2f m'%(left_curverad),(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,'Right radius of curvature  = %.2f m'%(right_curverad),(50,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,'Vehicle is %.2fm left of center'%(distance),(50,150), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        
    elif (vehicle_position - middle_of_the_lanes) > 0:
        cv2.putText(image,'Left radius of curvature  = %.2f m'%(left_curverad),(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,'Right radius of curvature  = %.2f m'%(right_curverad),(50,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,'Vehicle is %.2fm right of center'%(distance),(50,150), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
   
    else:
        cv2.putText(image,'Left radius of curvature  = %.2f m'%(left_curverad),(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,'Right radius of curvature  = %.2f m'%(right_curverad),(50,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,'Vehicle is Straight on the road',(50,150), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)

def detectorORtracker(state,left_fitx, right_fitx, left_fit, right_fit):
   #tracker
    if state==1:
        std_value = np.std(right_fitx - left_fitx)
        if std_value < 85:
            left_lane.detected = True
            right_lane.detected = True
            left_lane.current_poly = left_fit
            right_lane.current_poly = right_fit
            left_lane.cur_fitx = left_fitx
            right_lane.cur_fitx = right_fitx
            # global tt
            # tt = tt + 1
        else:
            left_lane.detected = False
            right_lane.detected = False
            left_lane.current_poly = left_lane.prev_poly
            right_lane.current_poly = right_lane.prev_poly
            left_lane.cur_fitx = left_lane.prev_fitx[-1]
            right_lane.cur_fitx = right_lane.prev_fitx[-1]
    #deterctor
    else:
        std_value = np.std(right_fitx - left_fitx)
        if std_value < 85:
            left_lane.current_poly = left_fit
            right_lane.current_poly = right_fit
            left_lane.cur_fitx = left_fitx
            right_lane.cur_fitx = right_fitx
            left_lane.detected = True
            right_lane.detected = True
        else:
            left_lane.current_poly = left_lane.prev_poly
            right_lane.current_poly = right_lane.prev_poly
            if len(left_lane.prev_fitx) > 0:
                left_lane.cur_fitx = left_lane.prev_fitx[-1]
                right_lane.cur_fitx = right_lane.prev_fitx[-1]
            else:
                left_lane.cur_fitx = left_fitx
                right_lane.cur_fitx = right_fitx
            left_lane.detected = False
            right_lane.detected = False   
    
def operation(image):     

    undistort_image = cv2.undistort(image, mtx, dist, None, mtx)

    ave_binary = combined(undistort_image)
    
    bird_view, Minv = warp(ave_binary)
    
    histogram = np.sum(bird_view[bird_view.shape[0]//2:,:], axis=0)
    
    left_fitx, right_fitx, ploty, left_fit, right_fit, leftx, rightx = cal_lane(histogram, bird_view)
    
    if left_lane.detected:  # start tracker
        detectorORtracker(1,left_fitx, right_fitx, left_fit, right_fit)
    else:                   # start detector
        detectorORtracker(2,left_fitx, right_fitx, left_fit, right_fit)
    
    left_lane.process(ploty)
    right_lane.process(ploty)
    
    cal_curvature(image, ploty, left_lane.mean_fitx, right_lane.mean_fitx)
    
    detected_image = draw_lanes(image, bird_view, left_lane.mean_fitx, right_lane.mean_fitx, ploty, Minv)
       
    return detected_image

img_for_calibration = glob.glob('/home/nsslab/SDCND/Project2/images_for_calibration/calibration*.jpg')
mtx, dist = calibration(img_for_calibration)


image = mpimg.imread('/home/nsslab/SDCND/Project2/test_images/test2.jpg')   
    
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'output_project_video.mp4'
clip1 = VideoFileClip("/home/nsslab/SDCND/Project2/project_video.mp4")
white_clip = clip1.fl_image(operation) 
%time white_clip.write_videofile(white_output , audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))



[MoviePy] >>>> Building video output_project_video.mp4
[MoviePy] Writing video output_project_video.mp4


100%|█████████▉| 1260/1261 [02:31<00:00,  8.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_project_video.mp4 

CPU times: user 8min 28s, sys: 12.4 s, total: 8min 40s
Wall time: 2min 31s
